In [1]:
import pandas as pd
import numpy as np
import duckdb
import os
import sys
import sys
sys.path.insert(0, os.path.abspath("../.."))
from utils import time, data_processor, constants

In [2]:
all_countries = set(constants.eu + constants.americas + constants.asia + constants.africa + constants.oceania)
eu = set(constants.eu)
balkans = set(constants.balkans)

In [3]:
INDUSTRY_CLASSIFICATION_PATH = "Z:\\dati_moody\\data_processed\\firmographics_processed\\industry_classification"
CONTACT_INFO_PATH = "Z:\\dati_moody\\data_processed\\firmographics_processed\\contact_info"
LEGAL_INFO_PATTERN = "../../data_raw/firmographics/legal_info/*.parquet"
ALL_ADDRESSES = "../../data_raw/firmographics/all_addresses/*.parquet"

## QUERY


In [4]:
# import duckdb

# INDUSTRY_CLASSIFICATION_PATH = "Z:\\dati_moody\\data_processed\\firmographics_processed\\industry_classification\\*.parquet"
# CONTACT_INFO_PATH = "Z:\\dati_moody\\data_processed\\firmographics_processed\\contact_info\\*.parquet"
# LEGAL_INFO_PATTERN = "Z:\\dati_moody\\data_raw\\firmographics\\legal_info\\*.parquet"

# OUTPUT_PATH = "Z:\\dati_moody\\data_processed\\firmographics_processed\\ita.parquet"
# OUTPUT_DIR = "Z:\\dati_moody\\data_processed\\firmographics_processed\\"  # Directory to save results

# countries = ["HR", "IT"]  # Example list of country codes

# for country in countries:
#     output_path = f"{OUTPUT_DIR}{country.lower()}.parquet"
#     query = f"""
#         COPY (
#             SELECT
#                 *
#             FROM
#                 read_parquet('{CONTACT_INFO_PATH}', union_by_name=true) as nace
#             LEFT JOIN
#                 read_parquet('{LEGAL_INFO_PATTERN}', union_by_name=true) as legal_info
#             ON
#                 nace."bvd_id_number" = legal_info."bvd_id_number"
#             WHERE
#                 nace."bvd_id_number" LIKE '{country}%'
#             LIMIT 100
#         ) TO '{output_path}' (FORMAT PARQUET);
#     """
#     conn.execute(query)
#     print(f"Results for {country} saved to {output_path}")


In [5]:
# import duckdb
# import os

# # Set up the temp directory path
# temp_dir = "data_processed/temp"
# os.makedirs(temp_dir, exist_ok=True)

# # Connect and configure DuckDB
# conn = duckdb.connect()
# conn.execute(f"PRAGMA temp_directory='{temp_dir}';")
# conn.execute("PRAGMA max_temp_directory_size='500GB';")


In [ ]:
import duckdb
import os
import time

temp_dir = "data_processed/temp"
os.makedirs(temp_dir, exist_ok=True)
INDUSTRY_CLASSIFICATION_PATH = "Z:\\dati_moody\\data_processed\\firmographics_processed\\industry_classification\\*.parquet"
CONTACT_INFO_PATH = "Z:\\dati_moody\\data_processed\\firmographics_processed\\contact_info\\*.parquet"
LEGAL_INFO_PATTERN = "Z:\\dati_moody\\data_processed\\firmographics_processed\\legal_info\\*.parquet"
ALL_ADDRESSES = "Z:\\dati_moody\\data_processed\\firmographics_processed\\all_addresses\\*.parquet"
OUTPUT_DIR = "Z:\\dati_moody\\data_processed\\firmographics_processed\\"
countries = ["HR", "IT"]

start_time = time.time()  # Start timer

# Connect to DuckDB
conn = duckdb.connect()
conn.execute(f"PRAGMA temp_directory='{temp_dir}';")
conn.execute("PRAGMA max_temp_directory_size='500GB';")

# Load data once
conn.execute(f"""
    CREATE OR REPLACE TEMPORARY TABLE nace AS
    SELECT * FROM read_parquet('{INDUSTRY_CLASSIFICATION_PATH}', union_by_name=true)
    ;
""")
print("Industry classification data loaded.")

conn.execute(f"""
    CREATE OR REPLACE TEMPORARY TABLE contact_info AS
    SELECT * FROM read_parquet('{CONTACT_INFO_PATH}', union_by_name=true)
    ;
""")
print("Contact info data loaded.")

conn.execute(f"""
    CREATE OR REPLACE TEMPORARY TABLE legal_info AS
    SELECT * FROM read_parquet('{LEGAL_INFO_PATTERN}', union_by_name=true)
    ;
""")
print('legal_info loaded')

conn.execute(f"""
    CREATE OR REPLACE TEMPORARY TABLE addresses AS
    SELECT * FROM read_parquet('{ALL_ADDRESSES}', union_by_name=true);
""")
print("addresse loaded")

elapsed_time = time.time() - start_time  # Calculate elapsed time
print(f"Total execution time: {elapsed_time:.2f} seconds")

import psutil
print(f"Memory used: {psutil.Process().memory_info().rss / 1e9:.2f} GB")


In [ ]:

# # Loop through countries and filter
# for country in all_countries:
#     output_path = f"{OUTPUT_DIR}/firmographics_full/{country}.parquet"
#     query = f"""
#         COPY (
#             SELECT
#                 *
#             FROM
#                 nace 
#             LEFT JOIN
#                 contact_info
#             ON
#                 nace.bvd_id_number = contact_info.bvd_id_number
#             LEFT JOIN
#                 legal_info
#             ON
#                 nace.bvd_id_number = legal_info.bvd_id_number
#             LEFT JOIN
#                 addresses
#             ON
#                 nace.bvd_id_number = addresses.bvd_id_number
#             WHERE
#                 nace.bvd_id_number LIKE '{country}%'
#             LIMIT 1000
#         )
#         TO '{output_path}' (FORMAT PARQUET);
#     """
#     conn.execute("PRAGMA max_temp_directory_size='100GB';")
#     print(f"Results for {country} saved to {output_path}")

# conn.close()

In [ ]:
def export_country_to_parquet(country_iso_code: str, output_path: str):
    query = f"""
        COPY (
            SELECT 
                nace.bvd_id_number,
                nace.nace_rev_2_core_code_4_digits_,
                contact.nuts2,
                legal_info.status,
                legal_info.type_of_entity,
                all_addresses.city_native_,
                all_addresses.postcode,
                all_addresses.country_iso_code,
                all_addresses.city_native_,
                all_addresses.city,
                all_addresses.region_in_country
            FROM
                '{INDUSTRY_CLASSIFICATION_PATH}' AS nace
            LEFT JOIN
                '{CONTACT_INFO_PATH}' AS contact
            ON
                nace.bvd_id_number = contact.bvd_id_number
            LEFT JOIN
                '{LEGAL_INFO_PATTERN}' AS legal_info
            ON
                nace.bvd_id_number = legal_info.bvd_id_number
            LEFT JOIN
                '{ALL_ADDRESSES}' AS all_addresses
            ON  
                all_addresses.bvd_id_number = nace.bvd_id_number
            WHERE
                all_addresses.country_iso_code = '{country_iso_code}'
                AND (
                    nace.nace_rev_2_core_code_4_digits_ IS NOT NULL 
                    OR contact.nuts2 IS NOT NULL 
                    OR legal_info.status IS NOT NULL 
                    OR legal_info.type_of_entity IS NOT NULL
                )
        )
        
        TO '{output_path}/firmographics_{country_iso_code}.parquet' (FORMAT 'parquet');
    """

    duckdb.sql(query)


In [ ]:
def export_country_to_parquet(country_iso_code: str, output_path: str):
    query = f"""
        COPY (
            SELECT 
                nace.bvd_id_number,
                nace.nace_rev_2_core_code_4_digits_,
                contact.nuts2,
                legal_info.status,
                legal_info.type_of_entity,
                addr.city_native_,
                addr.postcode,
                addr.country_iso_code,
                addr.city_native_,
                addr.city,
                addr.region_in_country
            FROM (
                SELECT * FROM '{ALL_ADDRESSES}' WHERE country_iso_code = '{country_iso_code}'
            ) AS addr
            LEFT JOIN '{INDUSTRY_CLASSIFICATION_PATH}' AS nace
                ON addr.bvd_id_number = nace.bvd_id_number
            LEFT JOIN '{CONTACT_INFO_PATH}' AS contact
                ON nace.bvd_id_number = contact.bvd_id_number
            LEFT JOIN '{LEGAL_INFO_PATTERN}' AS legal_info
                ON nace.bvd_id_number = legal_info.bvd_id_number
            WHERE
                nace.nace_rev_2_core_code_4_digits_ IS NOT NULL 
                OR contact.nuts2 IS NOT NULL 
                OR legal_info.status IS NOT NULL 
                OR legal_info.type_of_entity IS NOT NULL
        )
        TO '{output_path}/firmographics_{country_iso_code}.parquet' (FORMAT 'parquet');
    """
    duckdb.sql(query)

In [ ]:
# def export_country_to_parquet(country_iso_code: str, output_path: str):
#     query = f"""
#             SELECT 
#                 nace.bvd_id_number,
#                 nace.nace_rev_2_core_code_4_digits_,
#                 contact.nuts2,
#                 legal_info.status,
#                 legal_info.type_of_entity,
#                 all_addresses.city_native_,
#                 all_addresses.postcode,
#                 all_addresses.country_iso_code,
#                 all_addresses.city_native_,
#                 all_addresses.city,
#                 all_addresses.region_in_country
#             FROM
#                 '{INDUSTRY_CLASSIFICATION_PATH}' AS nace
#             LEFT JOIN
#                 '{CONTACT_INFO_PATH}' AS contact
#             ON
#                 nace.bvd_id_number = contact.bvd_id_number
#             LEFT JOIN
#                 '{LEGAL_INFO_PATTERN}' AS legal_info
#             ON
#                 nace.bvd_id_number = legal_info.bvd_id_number
#             LEFT JOIN
#                 '{ALL_ADDRESSES}' AS all_addresses
#             ON  
#                 all_addresses.bvd_id_number = nace.bvd_id_number
#             WHERE
#                 all_addresses.country_iso_code = '{country_iso_code}'
#                 AND (
#                     nace.nace_rev_2_core_code_4_digits_ IS NOT NULL 
#                     OR contact.nuts2 IS NOT NULL 
#                     OR legal_info.status IS NOT NULL 
#                     OR legal_info.type_of_entity IS NOT NULL
#                 )
#         LIMIT 100
        
#     """

#     duckdb.sql(query)


In [ ]:
output_path = "..\\..\\data_processed\\firmographics_processed\\"

In [ ]:
import pyarrow.parquet as pq

output_path = "..\\..\\data_processed\\firmographics_processed\\"
country_iso_code = 'IT'

query = f"""
    SELECT 
        nace.bvd_id_number,
        nace.nace_rev_2_core_code_4_digits_,
        contact.nuts2,
        legal_info.status,
        legal_info.type_of_entity,
        addr.city_native_,
        addr.postcode,
        addr.country_iso_code,
        addr.city_native_,
        addr.city,
        addr.region_in_country
    FROM (
        SELECT * FROM '{ALL_ADDRESSES}' WHERE country_iso_code = '{country_iso_code}'
    ) AS addr
    LEFT JOIN '{INDUSTRY_CLASSIFICATION_PATH}' AS nace
        ON addr.bvd_id_number = nace.bvd_id_number
    LEFT JOIN '{CONTACT_INFO_PATH}' AS contact
        ON nace.bvd_id_number = contact.bvd_id_number
    LEFT JOIN '{LEGAL_INFO_PATTERN}' AS legal_info
        ON nace.bvd_id_number = legal_info.bvd_id_number
    WHERE
        nace.nace_rev_2_core_code_4_digits_ IS NOT NULL 
        OR contact.nuts2 IS NOT NULL 
        OR legal_info.status IS NOT NULL 
        OR legal_info.type_of_entity IS NOT NULL
"""

output = f"{output_path}{country_iso_code}.parquet"
os.makedirs(output_path, exist_ok=True)

# Create a connection
con = duckdb.connect()

# # Execute query and get a RecordBatchReader
# reader = con.execute(query).fetch_record_batch_reader()

# # Write to parquet using pyarrow
# pq.write_table(reader, output, compression="snappy")




In [ ]:
country = 'CN' 
test =  export_country_to_parquet(country, output_path)

In [ ]:
# output_path = "Z:\\dati_moody\\data_processed\\firmographics_processed\\firmographics_CN.parquet"

for country in ['CN']:
    df = export_country_to_parquet(country)
    file_path = f"{output_path}firmographics_{country}.parquet"

    # Check if the file exists and remove it if necessary
    if os.path.exists(file_path):
        os.remove(file_path)

    # Write the DataFrame to parquet
    df.to_parquet(file_path)
    print(f"{country}")


In [ ]:
# TEMP_TABLE_KEY_FINANCIALS = "..\\..\\data_raw\\key_financials\\key_financials_eur\\*.parquet"

# def fetch_data_for_country_historical(country_iso_code):
#     query = f"""
#     SELECT 
#         number_of_employees, 
#         closing_date,
#         bvd_id_number,
#         total_assets,
#         operating_revenue_turnover_,
#         EXTRACT(YEAR FROM closing_date) AS year  
#     FROM 
#         '{TEMP_TABLE_KEY_FINANCIALS}' 
#     WHERE
#         bvd_id_number LIKE '{country_iso_code}%'
#         """
    
#     # Connect to DuckDB and execute the query   
#     conn = duckdb.connect()
#     df = conn.execute(query).fetchdf()
#     return df

In [ ]:
# ## Test
# # country = "IT"
# OUTPUT_PATH = "..\\..\\data_processed\\key_financials_processed\\"

# for country in all_countries:
#     df = fetch_data_for_country_historical(country)
#     os.makedirs(OUTPUT_PATH, exist_ok=True)
#     df.to_parquet(f"{OUTPUT_PATH}\\key_financials_processed_{country}.parquet")
#     print(f"{country}")
